#### Biopython: Entrez interface

Links
  * https://github.com/lanadominkovic/12-days-of-biopython/blob/main/12_days_of_biopython/day_02/day_02-accessing-ncbi-databases.ipynb
  * https://people.duke.edu/~ccc14/pcfb/biopython/BiopythonEntrez.html
https://biopython-tutorial.readthedocs.io/en/latest/notebooks/09%20-%20Accessing%20NCBIs%20Entrez%20databases.html
* https://www.youtube.com/watch?v=YPcyNSoucNs
* https://biopython-tutorial.readthedocs.io/en/latest/notebooks/09%20-%20Accessing%20NCBIs%20Entrez%20databases.html#EFetch:-Downloading-full-records-from-Entrez
* https://colab.research.google.com/drive/1KmsomBZD1VBqb3Y-Xg-mMjLawbAYsLMX?usp=sharing


Lots of sources
*  https://github.com/crazyhottommy/getting-started-with-genomics-tools-and-resources
https://github.com/hbctraining/Training-modules

 - Tips: 
   - always use an email.
   - no multiple requests (peak time 9AM-5PM ET)

task: efetch (download)
identifier: accession number, GI numer (id)
database: nuccore, medline, etc
output: rettype (fasta, genbank, etc)
display: retmode (text, xml, etc)

Dictionary {'key1':"value1",'key2':"value2, ...}
List []

using datasets 7906 sequences obtained
after seqkit 6000-10000 filter 336 sequences

In [ ]:
#%%capture #to omit output
#install biopython and upgrade pip (just in case)
%pip install --upgrade pip
%pip install biopython

In [5]:
#1. import modules
from Bio import Entrez, SeqIO, Medline, Seq
import numpy as np

#To use our email
Entrez.email= "K5426020@kadai.jp"
#Entrez.email = "A.N.Other@example.com"
Entrez.api_key = "4a2ea68aa6ce498eecead48bfb95bce4cc0a"

In [6]:
#To know which Biopython we have installed
import Bio
print(Bio.__version__)
#To get the list of databases
handle = Entrez.einfo() # get the information, which URL we used
print(handle)
records = Entrez.read(handle)
print(records) #Dictionary

print(records.keys()) #Key options available
print(records['DbList']) #List of databases or values for this key

#to know more about the database to make an advance query
handle=Entrez.einfo(db="nucleotide")
record=Entrez.read(handle)
handle.close
print(record.keys())
print('\n') #a line
print(record['DbInfo'])
print(record["DbInfo"]["Description"])
print("Description: ",record["DbInfo"]["Description"])
print(record["DbInfo"].keys())
print(record['DbInfo']['FieldList'])
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s,%(FullName)s, %(Description)s" % field)


1.81
{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}
dict_keys(['DbList'])
['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']
dict_keys(['DbInfo'])


{'DbName': 'nuccore', 'MenuName

In [2]:
#to know more about pubmed
handle=Entrez.einfo(db="pubmed")
record=Entrez.read(handle)
record["DbInfo"]["Description"]
record["DbInfo"].keys()
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s,%(FullName)s, %(Description)s" % field)

HTTPError: HTTP Error 500: Internal Server Error

In [4]:
#search in pubmed
handle=Entrez.esearch(db="pubmed", term="Enriquez-Vera")
record=Entrez.read(handle)
idlist=record["IdList"]
print(idlist)


#medline
handle=Entrez.efetch(db="pubmed", id=idlist, rettype="medline")
records=Medline.parse(handle)
records=list(records)
for record in records:
    print("title:", record.get("TI","?"))
    print("authors:",record.get("AU","?"))
    print("source:",record.get("SO","?"))
    print("")
#search among results
search_author="Valcarcel B"         
for record in records:
    if not "AU" in record:
        continue
    if search_author in record["AU"]:
        print("Author %s found: %s" % (search_author, record["SO"]))            

HTTPError: HTTP Error 500: Internal Server Error

In [4]:
#I will start my search for nucleotides

#6000:1000[SLEN] AND "Human T-cell leukemia virus type I"[ORGN] biomol_genomic[PROP] AND (PROP ORGN TITLHuman T-cell leukemia virus type I NCBI:txid11908

#download the list of genBank identifiers
handle=Entrez.esearch(db="nucleotide",term="Human T-cell leukemia virus type I[Orgn] AND 6000:10000[SLEN]")
record=Entrez.read(handle)
record["Count"] #total articles

#make a list
htlvlist=(record['IdList'])
print(htlvlist)
textfile=open("C:/Users/81805/OneDrive/vscode/practice_sequences_python/htlvlist.txt","w")
for item in htlvlist:
    textfile.write(item +"\n")
textfile.close()    
#download fasta
#handle=Entrez.efetch(db='nucleotide', id=htlvlist,rettype='fasta',retmode="text")
#record=SeqIO.parse(handle,'fasta')
#outputname="C:/Users/81805/OneDrive/vscode/practice_sequences_python/htlvlist.fasta"
#SeqIO.write(htlvlist,outputname,'fasta')


['2070973379', '1830623825', '1830623821', '1830623817', '1830623813', '1830623809', '1830623805', '1830623801', '1830623797', '1779419811', '1277357657', '1277357653', '1277357649', '1277357645', '1277357641', '1277357637', '1277357633', '1277357629', '1277357625', '1277357621']


In [81]:
Entrez.email= "enriquezv.dj@gmail.com"
handle=Entrez.esearch(db="nucleotide", term="HTLV-I")
record=Entrez.read(handle)
record["Count"] #total articles

HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
#How to download to a file
#to download with genbank
handle=Entrez.efetch(db='nuccore', id='34577062')
print(handle.read())

#to download with fasta
handle=Entrez.efetch(db='nuccore', id='34577062',rettype='fasta')
print(handle.read())

#nucleotide, accession number, genbank, text format
handle=Entrez.efetch(db='nuccore', id='NM_001126.2',rettype='gb', retmode='txt')
print(handle.read())

#Download and explore sequences in fasta
handle=Entrez.efetch(db='nucleotide', id='NM_001126.2',rettype='fasta', retmode='text')
record=SeqIO.read(handle,'fasta')
record.description
record.seq
len(record.seq)
print(record.seq)
#Download and explore sequences in gb
handle=Entrez.efetch(db='nucleotide', id='NM_001126.2',rettype='gb', retmode='text')
record=SeqIO.read(handle,'gb')
record.description
record.annotations
len(record.features)
print(record.features)
#multiple identifiers
handle=Entrez.efetch(db='nuccore', id='34577062, 186972394',rettype='fasta')
records=SeqIO.parse(handle,'fasta')
records=[i for i in records]
#output file genbank single sequence
handle=Entrez.efetch(db='nucleotide', id='NM_001126.2',rettype='gb')
record=SeqIO.read(handle,'gb')
outputname='/Users/denriquez/Library/CloudStorage/OneDrive-Personal/vscode/practice_sequences_python/adss1120.gb'
SeqIO.write(record,outputname,'gb')
#output file genbank single sequence
handle=Entrez.efetch(db='nucleotide', id='NM_001126.2',rettype='fasta')
record=SeqIO.read(handle,'fasta')
outputname='/Users/denriquez/Library/CloudStorage/OneDrive-Personal/vscode/practice_sequences_python/adss1120.fasta'
SeqIO.write(record,outputname,'fasta')
#output file multifasta
handle=Entrez.efetch(db='nucleotide', id='34577062,186972394',rettype='fasta')
record=SeqIO.parse(handle,'fasta')
outputname='/Users/denriquez/Library/CloudStorage/OneDrive-Personal/vscode/practice_sequences_python/twofiles.fasta'
SeqIO.write(record,outputname,'fasta')